In [ ]:
%pip install qiskit-algorithms
%pip install nbformat
%run ansatz.ipynb

ansatzes = [linear_ansatz, full_ansatz, circular_ansatz, pairwise_ansatz]
ansatz_labels = ['linear' , 'full', 'circular', 'pairwise']
ansatz_colors = ['blue' , 'orange', 'green', 'cyan']


In [ ]:
from qiskit.quantum_info import SparsePauliOp
from qiskit.visualization.array import array_to_latex

c0 = -0.80718
c1 = 0.17374
c2 = -0.23047
c3 = 0.12149
c4 = 0.16940
c5 = -0.04509
c6 = 0.04509
c7 = 0.16658
c8 = 0.17511

hamiltonian = SparsePauliOp.from_list([
    ("IIII", c0),
    ("ZIII", c1),
    ("ZZII", c2),
    ("IIZI", c1),
    ("IZZZ", c2),
    ("IZII", c3),
    ("ZIZI", c4),
    ("XZXI", c5),
    ("XIXZ", c6),
    ("XIXI", c6),
    ("XZXZ", c5),
    ("ZZZZ", c7),
    ("ZZZI", c7),
    ("ZIZZ", c8),
    ("IZIZ", c3),
])

# 2^4 --> 16 by 16 matrix
# array_to_latex(hamiltonian)

In [ ]:
from qiskit_algorithms.optimizers import *
import numpy as np
from qiskit.primitives import Estimator, Sampler
from qiskit_algorithms import VQE

MAXITER = 200

adam = ADAM(maxiter=MAXITER)
aqdg = AQGD(maxiter=MAXITER)
cg = CG(maxiter=MAXITER)
cobyla = COBYLA(maxiter=MAXITER)
l_bfgs_b = L_BFGS_B(maxiter=MAXITER)
# gsls = GSLS() problem with comparison
gradient_descent = GradientDescent(maxiter=MAXITER)
nelder_mead = NELDER_MEAD(maxiter=MAXITER)
nft = NFT(maxiter=MAXITER)
p_bfgs = P_BFGS()  # does not allow to specify maxiter, only max function evaluations
powell = POWELL(maxiter=MAXITER)
slsqp = SLSQP(maxiter=MAXITER)
spsa = SPSA(maxiter=MAXITER)
sampler = Sampler()
fid = QNSPSA.get_fidelity(circuit=ansatzes[0], sampler=Sampler())
qnspsa = QNSPSA(fid, maxiter=MAXITER)
tnc = TNC(maxiter=MAXITER)
umda = UMDA(maxiter=MAXITER)

# optimizers = [adam, aqdg, cg, cobyla, l_bfgs_b, gsls, gradient_descent, gradient_descent_state,
#   nelder_mead, nft, p_bfgs, powell, slsqp, spsa, qnspsa, tnc, scipy_optimizer, umda]
optimizers = [adam, aqdg, cg, cobyla, l_bfgs_b, gradient_descent,
              nelder_mead, nft, p_bfgs, powell, slsqp, spsa, qnspsa, tnc, umda]

# Set a seed for reproducibility
np.random.seed(16)

# Generate an array of 10 elements with random values in the range [-2π, 2π]
random_array = np.random.uniform(-2*np.pi, 2*np.pi, 16)

estimator = Estimator()
converge_counts = np.empty([len(optimizers), len(ansatzes)], dtype=object)
converge_vals = np.empty([len(optimizers), len(ansatzes)], dtype=object)

for i, optimizer in enumerate(optimizers):
    print("\rOptimizer: {}".format(type(optimizer).__name__), end="")

    for j, ansatz in enumerate(ansatzes):
        if type(optimizer).__name__ == 'QNSPSA':
            sampler = Sampler()
            fidelity = QNSPSA.get_fidelity(circuit=ansatz, sampler=sampler)

            optimizer = QNSPSA(fidelity, maxiter=MAXITER)

        counts = []
        values = []

        def store_intermediate_result(eval_count, parameters, mean, std):
            counts.append(eval_count)
            values.append(mean)

        vqe = VQE(estimator, ansatz, optimizer,
                  callback=store_intermediate_result, initial_point=random_array)
        result = vqe.compute_minimum_eigenvalue(operator=hamiltonian)
        converge_counts[i, j] = np.asarray(counts)
        converge_vals[i, j] = np.asarray(values)

## Caluclate reference value

In [ ]:
from qiskit_algorithms import NumPyMinimumEigensolver

numpy_solver = NumPyMinimumEigensolver()
result = numpy_solver.compute_minimum_eigenvalue(operator=hamiltonian)
ref_value = result.eigenvalue.real
print(f"Reference value: {ref_value:.5f}")

## MatPlot lib experimenting


In [ ]:
import matplotlib.pyplot as plt

rows = 5
cols = 3
fig, axs = plt.subplots(rows,cols, figsize=(15, 15))

i = 0
for r in range(rows):
    for c in range(cols):
        for j in range(len(ansatzes)):
          cnot_used = ansatzes[j].decompose().count_ops()['cx']
          ansatz_depth = ansatzes[j].decompose().depth()
          axs[r, c].plot(converge_counts[i][j],converge_vals[i,j], label=f'{ansatz_labels[j]} C={cnot_used} D={ansatz_depth}', color=ansatz_colors[j])
        axs[r, c].set_title(type(optimizers[i]).__name__)
        axs[r, c].axhline(y = ref_value, color = 'r', linestyle = '-')
        axs[r, c].legend(loc = "best")
        i += 1


fig.supxlabel('Eval count')
fig.supylabel('Energy')
fig.suptitle("Energy convergence for various optimizers (initial point = numpy random seed 17, maxiter = 200)")
fig.tight_layout() # Or equivalently,  "plt.tight_layout()"

## 3D experiment

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

fig, ax = plt.subplots(subplot_kw=dict(projection='3d'))
ax.invert_zaxis()
xx = np.arange(5,25,1)
yy = np.arange(5,25,1)
X, Y = np.meshgrid(xx,yy)
Z = np.full((len(X), len(X)), ref_value)
ax.plot_surface(X, Y , Z, color='red', alpha=0.3)
x = []
y = []
z = []
for j in range(len(ansatzes)):
    # Extract relevant data
    x.append(ansatzes[j].decompose().count_ops()['cx'])
    y.append(ansatzes[j].decompose().depth())
    z.append(converge_vals[0, j][-1])

ax.stem(x,y,z)





# Set labels and title
ax.set_xlabel('Circuit Depth')
ax.set_ylabel('CNOTs Used')
ax.set_zlabel('Energy Level')
ax.set_title("3D Histogram: Energy Convergence for Various Optimizers (Linear Ansatz)")
ax.set_xticks(x)
ax.set_yticks(y)
fig.tight_layout()

## Visualization of nuber of CNOTs and circuit depth

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

depth = []
cnots = []
z = []
fig, (ax1, ax2) = plt.subplots(2, 1)
fig.suptitle('A tale of 2 subplots')
for j in range(len(ansatzes)):
    # Extract relevant data
    cnots.append(ansatzes[j].decompose().count_ops()['cx'])
    depth.append(ansatzes[j].decompose().depth())
    z.append(converge_vals[0, j][-1])
    ax1.stem(ansatzes[j].decompose().count_ops()['cx'], converge_vals[0, j][-1], basefmt=" ", linefmt=ansatz_colors[j], label=ansatz_labels[j])
    ax2.stem(ansatzes[j].decompose().depth(), converge_vals[0, j][-1], basefmt=" ", linefmt=ansatz_colors[j], label=ansatz_labels[j])

# ax1.stem(cnots, z, basefmt=" ")
ax1.invert_yaxis()
ax1.set_ylabel('Energy level')
ax1.set_xlabel('CNOTs used')
ax1.axhline(y = ref_value, color = 'r', linestyle = '-')
ax1.set_xticks(cnots)


# ax2.stem(depth, z, basefmt=" ")
ax2.invert_yaxis()
ax2.set_xlabel('Circuit depth')
ax2.set_ylabel('Energy level')
ax2.axhline(y = ref_value, color = 'r', linestyle = '-')
ax2.set_xticks(depth)


fig.legend(*ax1.get_legend_handles_labels(),
            loc='upper center', ncol=4, bbox_to_anchor=(0.5, 0), bbox_transform=plt.gcf().transFigure)

plt.tight_layout()
plt.show()

In [ ]:
import matplotlib.pyplot as plt

fig = plt.figure(constrained_layout=True, figsize=(15, 18))

rows = 5
cols = 3
depth = []
cnots = []
energy = []
subfigs = fig.subfigures(rows, cols)


for i, subfig in enumerate(subfigs.flat):
    subfig.suptitle(type(optimizers[i]).__name__)

    axs = subfig.subplots(2, 1)
    ax1, ax2 = axs.flat

    for j in range(len(ansatzes)):
    # Extract relevant data
      cnots.append(ansatzes[j].decompose().count_ops()['cx'])
      depth.append(ansatzes[j].decompose().depth())
      energy.append(converge_vals[i, j][-1])
      ax1.stem(ansatzes[j].decompose().count_ops()['cx'], converge_vals[i, j][-1], basefmt=" ", linefmt=ansatz_colors[j], label=ansatz_labels[j])
      ax2.stem(ansatzes[j].decompose().depth(), converge_vals[i, j][-1], basefmt=" ", linefmt=ansatz_colors[j], label=ansatz_labels[j])

    ax1.invert_yaxis()
    ax1.set_xlabel('CNOTs used')
    ax1.axhline(y = ref_value, color = 'r', linestyle = '-')
    ax1.set_xticks(cnots)
    ax1.set_yticks(np.linspace(0,-2, 6))


    ax2.invert_yaxis()
    ax2.set_xlabel('Circuit depth')
    ax2.axhline(y = ref_value, color = 'r', linestyle = '-')
    ax2.set_xticks(depth)
    ax2.set_yticks(np.linspace(0,-2, 6))

    ax2.legend(*ax1.get_legend_handles_labels(),
            loc='lower center')


plt.show()